In [ ]:
#!/usr/bin/env python
# coding: utf-8
# author: Wanjun  HUANG
# data: July 4th, 2021 
import torch
import numpy as np
import scipy.io
import torch.nn as nn
import torch.nn.functional as F
import time
import matplotlib.pyplot as plt
import math 
import torch.utils.data as Data
from torch.autograd import Function
import gc
from scipy import sparse

global MAXMIN_V, MAXMIN_PQg,BRANFT,BUS_SLACK
global Ybus,baseMVA
global Real_Ptrain, Real_Qtrain, Real_Vmtrain, Real_Vatrain, Real_PQdtrain
global scale_vm, VmUb, VmLb, bus_slack, bus_PQg
global flagVm,flagVa,DELTA,flag_hisv,Nbus,Ntest

## whether there is GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
print("Let's use", torch.cuda.device_count(), "GPUs!")

## load system name
REPEAT = 1 #for speedup test: number of repeated computation 
s_epoch = 800 # minimum epoch for .pth model saving
p_epoch = 10 # print loss for each "p_epoch" epoch
model_version = 1 # version of model
Nbus = 300 # number of buses
sys_R = 2 # test case name
flag_test = 1 # 0-train model; 1-test well-trained model
flag_hisv = 1 # 1-use historical V to calculate dV;0-use predicted V to calculate dV
EpochVm = 2 # maximum epoch of Vm
EpochVa = 2 # maximum epoch of Va
batch_size_training = 100 # mini-batch size for training
batch_size_test = 1 # mini-batch size for test

## hyperparameters
Lrm = 1e-3 # learning rate for Vm
Lra = 1e-3 # learning rate for Va
k_dV = 1 # coefficient for dVa & dVm in post-processing
DELTA = 1e-4 # threshold of violation
scale_vm = torch.tensor([10]).float() # scaling of output Vm
scale_va = torch.tensor([10]).float() # scaling of output Va

# hidden layers for voltage magnitude (Vm) prediction
if Nbus == 300:
    khidden_Vm = np.array([8,6,4,2], dtype=int)
    khidden_Va = np.array([8,6,4,2], dtype=int)
    Neach = 12
else:
    khidden_Vm = np.array([8,4,2], dtype=int)
    khidden_Va = np.array([8,4,2], dtype=int) 
    Neach = 8000
    
Ntrain = int(4*Neach) 
Nsample = int(5*Neach)
Ntest = int(Neach)

#name of hidden layers for result saving
nmLm = 'Lm'
for i in range(khidden_Vm.shape[0]):
    nmLm = nmLm + str(khidden_Vm[i])
    
Lm = khidden_Vm.shape[0] # number of hidden layers

# hidden layers for voltage angles (Va) prediction
nmLa = 'La'
for i in range(khidden_Va.shape[0]):
    nmLa = nmLa + str(khidden_Va[i])    

La = khidden_Va.shape[0]  # number of hidden layers  

# results name
PATHVm = './modelvm'+str(Nbus)+'r'+str(sys_R)+'N'+str(model_version)+nmLm+'E'+str(EpochVm)+'.pth'
PATHVa = './modelva'+str(Nbus)+'r'+str(sys_R)+'N'+str(model_version)+nmLa+'E'+str(EpochVa)+'.pth'
PATHVms = './modelvm'+str(Nbus)+'r'+str(sys_R)+'N'+str(model_version)+nmLm
PATHVas = './modelva'+str(Nbus)+'r'+str(sys_R)+'N'+str(model_version)+nmLa
resultnm = './res_'+str(Nbus)+'r'+str(sys_R)+'M'+str(model_version)+'H'+str(flag_hisv)+ 'NT'+str(Ntrain) \
+'B'+str(batch_size_training) +'Em'+str(EpochVm)+'Ea'+str(EpochVa)+nmLm+nmLa+'rp'+str(REPEAT)+'.mat'

# load data case 300 large variance
mat = scipy.io.loadmat('./data/XY_case300real.mat')
load_idx = np.squeeze(mat['load_idx']).astype(int) - 1 
matpara = scipy.io.loadmat('./data/pglib_opf_case'+str(Nbus)+'_ieeer'+str(sys_R)+'_para.mat')

# power system parameters
RPd0 = mat['RPd'] # Pd
RQd0 = mat['RQd'] # Qd
RPg = mat['RPg'] # Pg
RQg = mat['RQg'] # Qg
Ybus = matpara['Ybus']
Yf = matpara['Yf']
Yt = matpara['Yt']
bus = matpara['bus']
gen = matpara['gen']
gencost = matpara['gencost']
branch = matpara['branch']
baseMVA = matpara['baseMVA']
bus_slack = np.where(bus[:, 1] == 3)
bus_slack = np.squeeze(bus_slack)
BUS_SLACK = torch.from_numpy(bus_slack).long()
print('bus_slack', bus_slack)

# numpy array to spase matrix
Ybus = sparse.csr_matrix(Ybus)
Yf = sparse.csr_matrix(Yf) 
Yt = sparse.csr_matrix(Yt) 

# output data
YVa = mat['RVa']*math.pi/180
# do not need to predict voltaeg angles of slack bus 
YVa = np.delete(YVa, bus_slack, axis=1) 
YVm = mat['RVm']
VmLb = mat['VmLb'] # lower bound of Vm
VmUb = mat['VmUb'] # upper bound of Vm
kvm = (YVm - VmLb)/(VmUb - VmLb) # scaled Vm

# output data
yva = YVa
yvm = kvm
print('yvm',yvm.shape,'yva',yva.shape)

In [ ]:
# input data: only contain non-zeros loads
idx_Pd = np.squeeze(np.where(np.abs(RPd0[0,:])> 0), axis=0)
idx_Qd = np.squeeze(np.where(np.abs(RQd0[0,:])> 0), axis=0)
bus_Pd = np.squeeze(load_idx[idx_Pd]).astype(int) - 1
bus_Qd = np.squeeze(load_idx[idx_Qd]).astype(int) - 1
# input data
x = np.concatenate((RPd0[:, idx_Pd], RQd0[:, idx_Qd]), axis = 1)/baseMVA 
print('x', x.shape)

In [ ]:
# Pg Qg data: only contain generators that are turned on
idxPg = np.squeeze(np.where(gen[:, 3] > 0), axis=0)
idxQg = np.squeeze(np.where(gen[:, 1] > 0), axis=0)

# Pd Qd of samples
RPd = np.zeros((Nsample,Nbus))
RQd = np.zeros((Nsample,Nbus))
RPd[:, load_idx] = RPd0[0:Nsample]
RQd[:, load_idx] = RQd0[0:Nsample]

del RPd0, RQd0
gc.collect()

In [ ]:
bus_Pg = gen[idxPg, 0].astype(int) - 1
bus_Qg = gen[idxQg, 0].astype(int) - 1
bus_PQg = np.concatenate((bus_Pg, bus_Qg + Nbus), axis = 0)
MAXMIN_Pg = gen[idxPg, 3:5]/baseMVA
MAXMIN_Qg = gen[idxQg, 1:3]/baseMVA
MAXMIN_PQg = np.concatenate((MAXMIN_Pg, MAXMIN_Qg), axis = 0) #[Npg+NQg, 2]

# numpy to tensor and output data scaling
MAXMIN_PQg = torch.from_numpy(MAXMIN_Pg).float()
bus = torch.from_numpy(bus).float()
bus_PQg = torch.from_numpy(bus_PQg)
VmLb = torch.from_numpy(VmLb).float()
VmUb = torch.from_numpy(VmUb).float()
yvm = torch.from_numpy(yvm).float()
yva = torch.from_numpy(yva).float()
yvms = yvm*scale_vm
yvas = yva*scale_va
# constraints violation: Vm 
MAXMIN_V = (bus[:, 2:4] - VmLb)*scale_vm/(VmUb - VmLb)
MAXMIN_Vpu = bus[0,2:4]

# branch angle incidence matrix
BRANFT = torch.from_numpy(branch[:, 0:2] - 1).long()

# loss function
criterion = nn.MSELoss()

# convert training data to tensor
x_tensor = torch.from_numpy(x).float()
yvm_tensor = yvms.float()
yva_tensor = yvas.float()
xtrain = x_tensor[0: Ntrain]
yvmtrain = yvm_tensor[0: Ntrain]
yvatrain = yva_tensor[0: Ntrain]

# batch data
training_dataset_vm = Data.TensorDataset(xtrain, yvmtrain)
training_loader_vm = Data.DataLoader(
        dataset=training_dataset_vm,
        batch_size=batch_size_training,
        shuffle=False,
    )             

training_dataset_va = Data.TensorDataset(xtrain, yvatrain)
training_loader_va = Data.DataLoader(
        dataset=training_dataset_va,
        batch_size=batch_size_training,
        shuffle=False,
    )

# test data
xtest = x_tensor[Ntrain: Nsample]
yvmtest = yvm_tensor[Ntrain: Nsample]
yvatest = yva_tensor[Ntrain: Nsample]

# batch data
batch_size_test = 1
test_dataset_vm = Data.TensorDataset(xtest, yvmtest)
test_loader_vm = Data.DataLoader(
        dataset=test_dataset_vm,
        batch_size=batch_size_test,
        shuffle=False,
    )             

test_dataset_va = Data.TensorDataset(xtest, yvatest)
test_loader_va = Data.DataLoader(
        dataset=test_dataset_va,
        batch_size=batch_size_test,
        shuffle=False,
    )

print('yvmtrain',torch.min(yvmtrain), torch.max(yvmtrain))
print('yvatrain',torch.min(yvatrain), torch.max(yvatrain))

# test load generation
Pdtest = RPd[Ntrain: Nsample]/baseMVA
Qdtest = RQd[Ntrain: Nsample]/baseMVA
Pgtest = RPg[Ntrain: Nsample, idxPg]/baseMVA
Qgtest = RQg[Ntrain: Nsample, idxQg]/baseMVA
Qgtest = Qgtest.squeeze()

# historical voltage
Real_Vmtrain = yvmtrain/scale_vm*(VmUb - VmLb) + VmLb
Real_Vatrain = yvatrain/scale_va
hisVm_max,_ = torch.max(Real_Vmtrain, dim=0)
hisVm_min,_ = torch.min(Real_Vmtrain, dim=0)
his_Va = np.mean(np.insert(Real_Vatrain.numpy(), bus_slack, values = 0, axis=1),axis=0)
his_Vm = np.mean(Real_Vmtrain.numpy(), axis=0)
his_V = his_Vm * np.exp(1j * his_Va) # historical V

In [ ]:
## other function
def get_clamp(Pred, Predmin, Predmax):
    # each row is a sample;Predmin and Predmax is the limit for each element of each row
    Pred_clip = Pred.clone()
    for i in range(Pred.shape[1]):
        Pred_clip[:,i] = Pred_clip[:,i].clamp(min = Predmin[i])
        Pred_clip[:,i] = Pred_clip[:,i].clamp(max = Predmax[i])
    
    return Pred_clip

# testing Vm
def get_mae(real, predict):
    '''
    mean absolute error
    '''
    if len(real) == len(predict):
        err = torch.mean(torch.abs(real - predict))  
        return err
    else:
        return None
    
def get_rerr(real, predict):
    '''
    relative error
    '''
    if len(real) == len(predict):
        err = torch.abs((predict - real) / real)*100
        return err
    else:
        return None
    
def get_rerr2(real, predict):
    '''
    relative error
    '''
    if len(real) == len(predict):
        err = (predict - real) / real*100
        return err
    else:
        return None
    
# power balance
def get_PQ(V):
    S = np.zeros(V.shape,dtype = 'complex_')
    for i in range(V.shape[0]):
        I = Ybus.dot(V[i]).conj()
        S[i]  = np.multiply(V[i], I)
    
    P = np.real(S)
    Q = np.imag(S) 
    return P, Q

def get_genload(V, Pdtest, Qdtest, bus_Pg, bus_Qg):
    S = np.zeros(V.shape,dtype = 'complex_')
    for i in range(V.shape[0]):
        I = Ybus.dot(V[i]).conj()
        S[i]  = np.multiply(V[i], I)
    
    P = np.real(S)
    Q = np.imag(S) 
    
    Pg = P[:, bus_Pg] + Pdtest[:, bus_Pg]
    Qg = Q[:, bus_Qg] + Qdtest[:, bus_Qg]   
    Pd = -P*1.0
    Qd = -Q*1.0
    Pd[:, bus_Pg] = Pg - P[:, bus_Pg]
    Qd[:, bus_Qg] = Qg - Q[:, bus_Qg]   
    return Pg, Qg, Pd, Qd


# cost
def get_Pgcost(Pg,idxPg,gencost):
    cost = np.zeros(Pg.shape[0])
    PgMVA = Pg*baseMVA
    PgMVA = Pg*baseMVA
    for i in range(Pg.shape[0]): 
        c1 = np.multiply(gencost[idxPg, 0], np.multiply(PgMVA[i,:], PgMVA[i,:]))
        c2 = np.multiply(gencost[idxPg, 1], PgMVA[i,:])
        cost[i] = np.sum(c1 + c2)
            
    return cost

In [ ]:
## NN function
class NetVa(nn.Module):
    def __init__(self, input_channels, output_channels, hidden_units,khidden):
        super(NetVa, self).__init__()
        #''' 
        self.num_layer = khidden.shape[0]
        
        self.fc1 = nn.Linear(input_channels, khidden[0]*hidden_units)  
        if self.num_layer >= 2: 
            self.fc2 = nn.Linear(khidden[0]*hidden_units, khidden[1]*hidden_units)
        
        if self.num_layer >= 3:
            self.fc3 = nn.Linear(khidden[1]*hidden_units, khidden[2]*hidden_units)
        
        if self.num_layer >= 4:
            self.fc4 = nn.Linear(khidden[2]*hidden_units, khidden[3]*hidden_units)
            
        if self.num_layer >= 5:
            self.fc5 = nn.Linear(khidden[3]*hidden_units, khidden[4]*hidden_units)
            
        if self.num_layer >= 6:
            self.fc6 = nn.Linear(khidden[4]*hidden_units, khidden[5]*hidden_units)
            
            
        self.fcbfend = nn.Linear(khidden[khidden.shape[0]-1]*hidden_units, output_channels)   
        self.fcend = nn.Linear(output_channels, output_channels)  

    def forward(self, x):
        x = F.relu(self.fc1(x))
        
        if self.num_layer >= 2:
            x = F.relu(self.fc2(x))
            
        if self.num_layer >= 3:
            x = F.relu(self.fc3(x))
            
        if self.num_layer >= 4:
            x = F.relu(self.fc4(x))
        
        if self.num_layer >= 5:
            x = F.relu(self.fc5(x))
            
        if self.num_layer >= 6:
            x = F.relu(self.fc6(x))
        
        # fixed final two layers
        x = F.relu(self.fcbfend(x))
        x_PredVa = self.fcend(x)
                
        return x_PredVa
        

class NetVm(nn.Module):
    def __init__(self, input_channels, output_channels, hidden_units,khidden):
        super(NetVm, self).__init__()
        self.num_layer = khidden.shape[0]
        self.fc1 = nn.Linear(input_channels, khidden[0]*hidden_units)
        if self.num_layer >= 2: 
            self.fc2 = nn.Linear(khidden[0]*hidden_units, khidden[1]*hidden_units)
        
        if self.num_layer >= 3:
            self.fc3 = nn.Linear(khidden[1]*hidden_units, khidden[2]*hidden_units)
        
        if self.num_layer >= 4:
            self.fc4 = nn.Linear(khidden[2]*hidden_units, khidden[3]*hidden_units)
            
        if self.num_layer >= 5:
            self.fc5 = nn.Linear(khidden[3]*hidden_units, khidden[4]*hidden_units)
            
        if self.num_layer >= 6:
            self.fc6 = nn.Linear(khidden[4]*hidden_units, khidden[5]*hidden_units)
            
            
        self.fcbfend = nn.Linear(khidden[khidden.shape[0]-1]*hidden_units, output_channels)   
        self.fcend = nn.Linear(output_channels, output_channels) 
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        
        if self.num_layer >= 2:
            x = F.relu(self.fc2(x))
            
        if self.num_layer >= 3:
            x = F.relu(self.fc3(x))
            
        if self.num_layer >= 4:
            x = F.relu(self.fc4(x))
        
        if self.num_layer >= 5:
            x = F.relu(self.fc5(x))
            
        if self.num_layer >= 6:
            x = F.relu(self.fc6(x))
        
        # fixed final two layers
        x = F.relu(self.fcbfend(x))
        x_PredVm = self.fcend(x)

        return x_PredVm

In [ ]:
## violation calculation function
# Pg Qg violation
def get_vioPQg(Pred_Pg, bus_Pg, MAXMIN_Pg, Pred_Qg, bus_Qg, MAXMIN_Qg):
    vio_PQgmaxminnum = torch.zeros((Pred_Pg.shape[0],4))
    vio_PQgmaxmin = torch.zeros((Pred_Pg.shape[0],4))
    vio_PQg = torch.zeros((Pred_Pg.shape[0],2))
    lsPg = list()
    lsQg = list()
    lsidxPg = np.zeros((Pred_Pg.shape[0]),dtype = np.int) # index of sample that has violation
    lsidxQg = np.zeros((Pred_Pg.shape[0]),dtype = np.int) # index of sample that has violation
    kP = 1 # violated samples index
    kQ = 1 # violated samples index
    deltaPgL = np.array([[0, 0]])
    deltaPgU = np.array([[0, 0]])
    deltaQgL = np.array([[0, 0]])
    deltaQgU = np.array([[0, 0]])
    for i in range(Pred_Pg.shape[0]):
        # P
        delta = Pred_Pg[i] - MAXMIN_Pg[:, 0]
        idxPgUB = np.array(np.where(delta > DELTA))
        if np.size(idxPgUB) > 0:
            PgUB = np.concatenate((idxPgUB,delta[idxPgUB]),axis=0).T 
            deltaPgU = np.append(deltaPgU, PgUB, axis = 0)

        delta = Pred_Pg[i] - MAXMIN_Pg[:, 1]
        idxPgLB = np.array(np.where(delta < -DELTA))
        if np.size(idxPgLB) > 0:
            PgLB = np.concatenate((idxPgLB,delta[idxPgLB]),axis=0).T
            deltaPgL = np.append(deltaPgL, PgLB, axis = 0)
        
        if np.size(idxPgUB)>0 and np.size(idxPgLB) > 0:
            PgLUB = np.concatenate((PgUB,PgLB),axis=0)
        elif np.size(idxPgUB) > 0:
            PgLUB = PgUB
        elif np.size(idxPgLB) > 0:
            PgLUB = PgLB
        
        if (np.size(idxPgUB) + np.size(idxPgLB)) > 0:
            PgLUB = PgLUB[PgLUB[:,0].argsort()]
            lsPg.append(PgLUB)
            lsidxPg[i] = kP # has violation:index of lsPg 
            kP+= 1#index=lsPg

        # Q
        delta = Pred_Qg[i] - MAXMIN_Qg[:, 0]
        idxQgUB = np.array(np.where(delta > DELTA))
        if np.size(idxQgUB) > 0:
            QgUB = np.concatenate((idxQgUB,delta[idxQgUB]),axis=0).T
            deltaQgU = np.append(deltaQgU, QgUB, axis = 0)

        delta = Pred_Qg[i] - MAXMIN_Qg[:, 1]
        idxQgLB = np.array(np.where(delta < -DELTA))
        if np.size(idxQgLB) > 0:
            QgLB = np.concatenate((idxQgLB,delta[idxQgLB]),axis=0).T
            deltaQgL = np.append(deltaQgL, QgLB, axis = 0)
            
        if np.size(idxQgUB) >0 and np.size(idxQgLB) > 0:   
            QgLUB = np.concatenate((QgUB,QgLB),axis=0)
        elif np.size(idxQgUB) > 0:
            QgLUB = QgUB
        elif np.size(idxQgLB) > 0:
            QgLUB = QgLB
         
        if (np.size(idxQgUB) + np.size(idxQgLB)) > 0:
            QgLUB = QgLUB[QgLUB[:,0].argsort()]
            lsQg.append(QgLUB)
            lsidxQg[i] = kQ # has violation
            kQ+= 1
                    
        
        vio_PQgmaxminnum[i,0] = np.size(idxPgUB)
        vio_PQgmaxminnum[i,1] = np.size(idxPgLB)
        vio_PQgmaxminnum[i,2] = np.size(idxQgUB)
        vio_PQgmaxminnum[i,3] = np.size(idxQgLB)
        
    # Pg Qg violation ratio
    vio_PQgmaxmin[:,0] = (1 - vio_PQgmaxminnum[:, 0]/bus_Pg.shape[0])*100
    vio_PQgmaxmin[:,1] = (1 - vio_PQgmaxminnum[:, 1]/bus_Pg.shape[0])*100
    vio_PQgmaxmin[:,2] = (1 - vio_PQgmaxminnum[:, 2]/bus_Qg.shape[0])*100
    vio_PQgmaxmin[:,3] = (1 - vio_PQgmaxminnum[:, 3]/bus_Qg.shape[0])*100
    vio_PQg[:, 0] = (1 - (vio_PQgmaxminnum[:, 0] + vio_PQgmaxminnum[:, 1])/bus_Pg.shape[0])*100
    vio_PQg[:, 1] = (1 - (vio_PQgmaxminnum[:, 2] + vio_PQgmaxminnum[:, 3])/bus_Qg.shape[0])*100
     
    # delete initial 
    if deltaPgL.shape[0] > 1:
        deltaPgL = np.delete(deltaPgL, 0, axis = 0)
        
    if deltaPgU.shape[0] > 1:    
        deltaPgU = np.delete(deltaPgU, 0, axis = 0)
    
    if deltaQgL.shape[0] > 1:
        deltaQgL = np.delete(deltaQgL, 0, axis = 0)
    
    if deltaQgU.shape[0] > 1:
        deltaQgU = np.delete(deltaQgU, 0, axis = 0)
    
    return lsPg,lsQg,lsidxPg,lsidxQg,vio_PQgmaxmin,vio_PQg, deltaPgL,deltaPgU,deltaQgL,deltaQgU

In [ ]:
# modify V theta
# dPbus_dV dQbus_dV
def dPQbus_dV(his_V,bus_Pg,bus_Qg):
    V = his_V.copy()
#     Ibus = np.dot(Ybus, his_V)
    Ibus = Ybus.dot(his_V).conj()
    diagV = np.diag(V)
    diagIbus = np.diag(Ibus)
    diagVnorm = np.diag(V/np.abs(V))
    dSbus_dVm = np.dot(diagV, Ybus.dot(diagVnorm).conj()) + np.dot(diagIbus.conj(), diagVnorm)
    dSbus_dVa = 1j*np.dot(diagV, (diagIbus - Ybus.dot(diagV)).conj())
    dSbus_dV = np.concatenate((dSbus_dVa, dSbus_dVm), axis=1)
    dPbus_dV = np.real(dSbus_dV)
    dQbus_dV = np.imag(dSbus_dV)
    
    return dPbus_dV,dQbus_dV

# calculate dV using historical V
def get_hisdV(lsPg,lsQg,lsidxPg,lsidxQg,num_viotest,k_dV,bus_Pg,bus_Qg,dPbus_dV,dQbus_dV):
    dV = np.zeros((num_viotest,Nbus*2))
    j = 0
    for i in range(Ntest):
        # determin whether there is violation
        if (lsidxPg[i] + lsidxQg[i]) > 0:
            # calculate dS_dV, dPbus_dV,dQbus_dV
            if lsidxPg[i] >0 and lsidxQg[i] >0:
                idxPg = lsPg[lsidxPg[i]-1][:,0].astype(np.int32) #note: lsidxPg[i]-1
                idxQg = lsQg[lsidxQg[i]-1][:,0].astype(np.int32) #note: lsidxQg[i]-1
                busPg = bus_Pg[idxPg]
                busQg = bus_Qg[idxQg]
                dPQGbus_dV = np.concatenate((dPbus_dV[busPg, :], dQbus_dV[busQg, :]), axis=0) #need bus number pf Pg Qg
                dPQg = np.concatenate((lsPg[lsidxPg[i]-1][:,1], lsQg[lsidxQg[i]-1][:,1]), axis=0)
            elif lsidxPg[i] >0:  
                idxPg = lsPg[lsidxPg[i]-1][:,0].astype(np.int32)
                busPg = bus_Pg[idxPg]
                dPQGbus_dV = dPbus_dV[busPg, :]
                dPQg = lsPg[lsidxPg[i]-1][:,1]
            elif lsidxQg[i] >0:     
                idxQg = lsQg[lsidxQg[i]-1][:,0].astype(np.int32)
                busQg = bus_Qg[idxQg]
                dPQGbus_dV = dQbus_dV[busQg, :]
                dPQg = lsQg[lsidxQg[i]-1][:,1]

            dV[j] = np.dot(np.linalg.pinv(dPQGbus_dV), dPQg*k_dV)           
            j+=1   
            
    return dV

# # calculate dV using predicted V
# lsidxtest is the test sample that has violation
def get_dV(Pred_V,lsPg,lsQg,lsidxPg,lsidxQg,num_viotest,k_dV,bus_Pg,bus_Qg):
    dV = np.zeros((num_viotest,Pred_V.shape[1]*2))
    j = 0
    for i in range(Pred_V.shape[0]):
        # determin whether there is violation
        if (lsidxPg[i] + lsidxQg[i]) >0:
            # dS_dV
            V = Pred_V[i].copy()
            Ibus = Ybus.dot(his_V).conj()
            diagV = np.diag(V)
            diagIbus = np.diag(Ibus)
            diagVnorm = np.diag(V/np.abs(V))

            dSbus_dVm = np.dot(diagV, Ybus.dot(diagVnorm).conj()) + np.dot(diagIbus.conj(), diagVnorm)
            dSbus_dVa = 1j*np.dot(diagV, (diagIbus - Ybus.dot(diagV)).conj())
            
            dSbus_dV = np.concatenate((dSbus_dVa, dSbus_dVm), axis=1)
            dPbus_dV = np.real(dSbus_dV)
            dQbus_dV = np.imag(dSbus_dV)
            # get BusPg
            if lsidxPg[i] >0 and lsidxQg[i] >0:
                idxPg = lsPg[lsidxPg[i]-1][:,0].astype(np.int32) #note: lsidxPg[i]-1
                idxQg = lsQg[lsidxQg[i]-1][:,0].astype(np.int32) #note: lsidxQg[i]-1
                busPg = bus_Pg[idxPg]
                busQg = bus_Qg[idxQg]
                dPQGbus_dV = np.concatenate((dPbus_dV[busPg, :], dQbus_dV[busQg, :]), axis=0) #need bus number pf Pg Qg
                dPQg = np.concatenate((lsPg[lsidxPg[i]-1][:,1], lsQg[lsidxQg[i]-1][:,1]), axis=0)
            elif lsidxPg[i] >0:  
                idxPg = lsPg[lsidxPg[i]-1][:,0].astype(np.int32)
                busPg = bus_Pg[idxPg]
                dPQGbus_dV = dPbus_dV[busPg, :]
                dPQg = lsPg[lsidxPg[i]-1][:,1]
            elif lsidxQg[i] >0:     
                # get BusQg
                idxQg = lsQg[lsidxQg[i]-1][:,0].astype(np.int32)
                busQg = bus_Qg[idxQg]
                dPQGbus_dV = dQbus_dV[busQg, :]
                dPQg = lsQg[lsidxQg[i]-1][:,1]

            dV[j] = np.dot(np.linalg.pinv(dPQGbus_dV), dPQg*k_dV)           
            j+=1        
    return dV

In [ ]:
# branch constraints
def get_viobran(Pred_V,Pred_Va,branch,Yf,Yt):
   #branch format: [branch from bus, branch to bus, branch limit, minang, maxang]
    branlp = branch[:, 2]/baseMVA
    angminmax = branch[:, 3:5]*math.pi/180
    Pred_branang = Pred_Va[:, BRANFT[:, 0]] - Pred_Va[:, BRANFT[:, 1]]
    vio_branangnum = torch.zeros(Pred_V.shape[0])
    vio_branpfnum = torch.zeros(Pred_V.shape[0])
    deltapf = np.array([[0, 0]])
    for i in range(Pred_V.shape[0]):
    #for i in range(1):
        vio_branangnum[i] = np.size(np.where(Pred_branang[i, :] - angminmax[:,0] < -DELTA)) \
        + np.size(np.where(Pred_branang[i, :] - angminmax[:,1] > DELTA))

        # branch power flow
        fV = Pred_V[i, BRANFT[:, 0]]
        tV = Pred_V[i, BRANFT[:, 1]]
        fI = Yf.dot(Pred_V[i]).conj()
        tI = Yt.dot(Pred_V[i]).conj()       
        fS = np.multiply(fV, fI)
        tS = np.multiply(tV, tI)
        deltafS = np.abs(fS) - branlp
        deltatS = np.abs(tS) - branlp
        deltafS = np.array(deltafS).ravel()
        deltatS = np.array(deltatS).ravel()
        idxfs = np.array(np.where(deltafS > DELTA))
        idxts = np.array(np.where(deltatS > DELTA))
        vio_branpfnum[i] = np.size(idxfs) + np.size(idxfs)
        
        # save violated samples [index-of-branch violation-degree]
        if np.size(idxfs) >= 1:
            ii = np.concatenate((idxfs,deltafS[idxfs]),axis=0)
            deltapf = np.append(deltapf, ii.T, axis = 0)
            
        if np.size(idxts) >= 1:
            ii = np.concatenate((idxts,deltatS[idxts]),axis=0)
            deltapf = np.append(deltapf, ii.T, axis = 0) 
            
    # delete initial
    if deltapf.shape[0] > 1:
        deltapf = np.delete(deltapf, 0, axis = 0)
        deltapfR = deltapf[:, 1]/branlp[0,deltapf[:, 0].astype(int)]*100
        deltapf = np.insert(deltapf, 2, values=deltapfR, axis=1)
 
    vio_branang = (1-vio_branangnum/branch.shape[0])*100 
    vio_branpf = (1-vio_branpfnum/(branch.shape[0]*2))*100
    return vio_branang,vio_branpf,deltapf

In [ ]:
# branch constraints with violation details for dV
def get_viobran2(Pred_V,Pred_Va,branch,Yf,Yt):
   #branch=[branch from bus, branch to bus, branch limit, minang, maxang]
    branlp = branch[:, 2]/baseMVA
    angminmax = branch[:, 3:5]*math.pi/180
    Pred_branang = Pred_Va[:, BRANFT[:, 0]] - Pred_Va[:, BRANFT[:, 1]]
    vio_branangnum = torch.zeros(Pred_V.shape[0])
    vio_branpfnum = torch.zeros(Pred_V.shape[0])
    vio_branpfidx = torch.zeros(Pred_V.shape[0])
    lsSf = []
    lsSt = []
    lsSf_sampidx = []
    lsSt_sampidx = []
    deltapf = np.array([[0, 0]])
    for i in range(Pred_V.shape[0]):
    #for i in range(1):
        vio_branangnum[i] = np.size(np.where(Pred_branang[i, :] - angminmax[:,0] < -DELTA)) \
        + np.size(np.where(Pred_branang[i, :] - angminmax[:,1] > DELTA))

        # branch power flow
        fV = Pred_V[i, BRANFT[:, 0]]
        tV = Pred_V[i, BRANFT[:, 1]]        
        fI = Yf.dot(Pred_V[i]).conj()
        tI = Yt.dot(Pred_V[i]).conj()
        fS = np.multiply(fV, fI)
        tS = np.multiply(tV, tI)
        deltafS = np.abs(fS) - branlp
        deltatS = np.abs(tS) - branlp
        deltafS = np.array(deltafS).ravel()
        deltatS = np.array(deltatS).ravel()
        idxfs = np.array(np.where(deltafS > DELTA)).reshape(-1, 1)
        idxts = np.array(np.where(deltatS > DELTA)).reshape(-1, 1)
        vio_branpfnum[i] = np.size(idxfs) + np.size(idxfs)
        if np.size(idxfs) >= 1:           
            ii = np.concatenate((idxfs,deltafS[idxfs]),axis=1)  
            deltapf = np.append(deltapf, ii, axis = 0) 
            ii = np.concatenate((ii,np.real(fS[idxfs]),np.imag(fS[idxfs])),axis=1)
            lsSf.append(ii)
            lsSf_sampidx.append(i)           
            
        if np.size(idxts) >= 1:
            ii = np.concatenate((idxts,deltatS[idxts]),axis=1)
            deltapf = np.append(deltapf, ii, axis = 0) 
            ii = np.concatenate((ii,np.real(tS[idxts]),np.imag(tS[idxts])),axis=1)
            lsSt.append(ii)
            lsSt_sampidx.append(i)

        if np.size(idxfs) + np.size(idxts) >= 1:
            vio_branpfidx[i] = i+1
            
    # delete initial
    if deltapf.shape[0] > 1:
        deltapf = np.delete(deltapf, 0, axis = 0)
        deltapfR = deltapf[:, 1]/branlp[deltapf[:, 0].astype(int)]*100
        deltapf = np.insert(deltapf, 2, values=deltapfR, axis=1)
 
    vio_branang = (1-vio_branangnum/branch.shape[0])*100 
    vio_branpf = (1-vio_branpfnum/(branch.shape[0]*2))*100
    return vio_branang,vio_branpf,deltapf,vio_branpfidx,lsSf,lsSt,lsSf_sampidx,lsSt_sampidx

def dSlbus_dV(his_V,bus_Va):
    V = his_V.copy()
    branlp = branch[:, 2]/baseMVA
    # branch power flow for FROM branch
    fV = V[BRANFT[:, 0]]
    fI = Yf.dot(V).conj()
    Nsam = 1
    dfP_dVm = np.zeros((Nsam, branch.shape[0], Nbus))   
    dfQ_dVm = np.zeros((Nsam, branch.shape[0], Nbus))
    dfP_dVa = np.zeros((Nsam, branch.shape[0], Nbus-1))    
    dfQ_dVa = np.zeros((Nsam, branch.shape[0], Nbus-1))                
    diagfI = np.diag(fI)
    diagfV = np.diag(fV)
    diagVnorm = np.diag(np.true_divide(V, np.abs(V)))
#     print('diagfV',diagfV.shape,'diagVnorm',diagVnorm.shape,'Yf',Yf.shape,'diagfI',diagfI.shape,'finc',finc.shape)
    dfS_dVm = np.dot(diagfV, Yf.dot(diagVnorm).conj()) + np.dot(diagfI.conj(), np.dot(finc,diagVnorm))
    dfP_dVm = np.real(dfS_dVm)
    dfQ_dVm = np.imag(dfS_dVm)
    diagV = np.diag(V)
    dfS_dVa = -1j*np.dot(diagfV, Yf.dot(diagV).conj()) + 1j*np.dot(diagfI.conj(), np.dot(finc, diagV))
    dfP_dVa = np.real(dfS_dVa[:, bus_Va])
    dfQ_dVa = np.imag(dfS_dVa[:, bus_Va])
    dPfbus_dV = np.concatenate((dfP_dVa, dfP_dVm), axis = 1)
    dQfbus_dV = np.concatenate((dfQ_dVa, dfQ_dVm), axis = 1)

#     for TO branch
#     fI = np.zeros((V.shape[0], branch.shape[0]), dtype=complex)
#     tI = np.zeros((V.shape[0], branch.shape[0]), dtype=complex)
#     tV = V[0, BRANFT[:, 1]]
#     tI = Yt.dot(Yt, V).conj()
#     tS  = np.multiply(tV, tI)    
#     diagtI = np.diag(tI)
#     diagtV = np.diag(tV)
#     dtP_dVm = np.zeros((Nsam, branch.shape[0], Nbus))
#     dtQ_dVm = np.zeros((Nsam, branch.shape[0], Nbus))
#     dtP_dVa = np.zeros((Nsam, branch.shape[0], Nbus-1))
#     dtQ_dVa = np.zeros((Nsam, branch.shape[0], Nbus-1))
#     dtS_dVm = np.dot(diagtV, Yt.dot(diagVnorm).conj()) + np.dot(diagtI.conj(), np.dot(tinc,diagVnorm))
#     dtS_dVa = -1j*np.dot(diagtV, Yt.dot(diagV).conj()) + 1j*np.dot(diagtI.conj(), np.dot(tinc, diagV))
#     dtP_dVm = np.real(dtS_dVm)
#     dtQ_dVm = np.imag(dtS_dVm)
#     dtP_dVa = np.real(dtS_dVa[:, bus_Va])
#     dtQ_dVa = np.imag(dtS_dVa[:, bus_Va])

    return dPfbus_dV, dQfbus_dV

In [ ]:
# neural setting
input_channels = xtrain.shape[1]
output_channels_vm = yvmtrain.shape[1]
output_channels_va = yvatrain.shape[1]
lossvm = [] # save training losses of Vm
lossva = [] # save training losses of Va

# determine size of hidden layers
if x.shape[1] >= 100:
    hidden_units = 128
elif x.shape[1] > 30:
    hidden_units = 64
else:
    hidden_units = 16

In [ ]:
# train model if it is not test
if flag_test == 0:
    model_vm = NetVm(input_channels,output_channels_vm,hidden_units,khidden_Vm)
    optimizer_vm = torch.optim.Adam(model_vm.parameters(), lr=Lrm)

    if torch.cuda.is_available():
        model_vm.to(device)
        print('model_vm.to(device)')

    MAXMIN_V = MAXMIN_V.to(device)
    print('*' * 5+'Vm training'+'*' * 5)
    #Training process: Voltage magnitude
    start_time = time.process_time()
    for epoch in range(EpochVm):
        running_loss = 0.0
        for step, (train_x, train_y) in enumerate(training_loader_vm):
            #feedforward
            train_x, train_y= train_x.to(device), train_y.to(device)
            yvmtrain_hat = model_vm(train_x)    

            # if epoch less than specified number/no penalty of V: only MSEloss
            loss = criterion(train_y, yvmtrain_hat)
            running_loss =  running_loss + loss.item()
            
            # backproprogate
            optimizer_vm.zero_grad()
            loss.backward()
            optimizer_vm.step()      

        lossvm.append(running_loss)

        if (epoch+1)%p_epoch == 0:          
            print('epoch', epoch+1, running_loss,torch.min(yvmtrain_hat).detach(),torch.max(yvmtrain_hat).detach())
         
        # save trianed model
        if (epoch+1)%100 == 0 and (epoch+1) >= s_epoch:            
            torch.save(model_vm.state_dict(), PATHVms+'E'+str(epoch+1)+'F'+str(flagVm)+'.pth',_use_new_zipfile_serialization=False)
            
    time_trianVm = time.process_time()-start_time  
    print("\n")        
    print('time_trianVm',round(time_trianVm,5),'seconds')
    print("\n")
    
    # save trianed model
    torch.save(model_vm.state_dict(), PATHVm,_use_new_zipfile_serialization=False)
    # plot training loss 
    fig = plt.figure()
    plt.plot(lossvm)
    plt.title('lossvm')

In [ ]:
# testing Vm using training data
if flag_test == 0:
    print('test training data')
    xtrain = xtrain.to(device) 
    yvmtrain_hat = model_vm(xtrain)
    yvmtrain_hat = yvmtrain_hat.cpu()
    yvmtrains = yvmtrain/scale_vm*(VmUb - VmLb) + VmLb
    yvmtrain_hats = yvmtrain_hat.detach()/scale_vm*(VmUb - VmLb) + VmLb
    yvmtrain_hat_clip = get_clamp(yvmtrain_hats, hisVm_min, hisVm_max)

    mae_Vmtrain = get_mae(yvmtrains, yvmtrain_hat_clip.detach())
    mre_Vmtrain = get_rerr(yvmtrains,yvmtrain_hats.detach())
    mre_Vmtrain_clip = get_rerr(yvmtrains,yvmtrain_hat_clip.detach())

In [ ]:
if flag_test == 0:
    #Training process: voltage angle
    model_va = NetVa(input_channels,output_channels_va,hidden_units,khidden_Vm)
    optimizer_va = torch.optim.Adam(model_va.parameters(), lr=Lra)
    model_va.to(device)
    print('model_va.to(device)')
    
    start_time = time.process_time()
    for epoch in range(EpochVa):
        running_loss = 0.0
        for step, (train_x, train_y) in enumerate(training_loader_va):
            #feedforward
            train_x, train_y = train_x.to(device), train_y.to(device)
            yvatrain_hat = model_va(train_x)
            loss = criterion(train_y, yvatrain_hat)
            running_loss =  running_loss + loss.item()

            # backproprogate
            optimizer_va.zero_grad()
            loss.backward()
            optimizer_va.step()

        lossva.append(running_loss)

        if (epoch+1)%p_epoch == 0:          
            print('epoch', epoch+1, running_loss, torch.min(yvatrain_hat).detach(),torch.max(yvatrain_hat).detach())

        # save trianed model
        if (epoch+1)%100 == 0 and (epoch+1)>= s_epoch:           
            torch.save(model_va.state_dict(), PATHVas +'E'+str(epoch+1)+'F'+str(flagVa)+'.pth',_use_new_zipfile_serialization=False)

    time_trianVa = time.process_time() - start_time  
    
    print("\n")        
    print('time_trianVa',round(time_trianVa,5), 'seconds')
    print("\n")  
    
    # save trianed model
    torch.save(model_va.state_dict(), PATHVa, _use_new_zipfile_serialization=False)
    fig = plt.figure()
    plt.plot(lossva)
    plt.title('lossva')

In [ ]:
# testing Va using training data
if flag_test == 0:
    print('test trianing data')
    xtrain = xtrain.to(device)
    yvatrain_hat = model_va(xtrain)
    yvatrain_hat = yvatrain_hat.cpu()
    yvatrains = yvatrain/scale_va
    yvatrain_hats = yvatrain_hat.detach()/scale_va

    mae_Vatrain = get_mae(yvatrains, yvatrain_hats)
    mre_Vatrain = get_rerr(yvatrains,yvatrain_hats)

In [ ]:
## save mat data
scale_vm = np.double(scale_vm)
scale_va = np.double(scale_va)

xtest = xtest.to(device)  

# Real_Vatrain = np.insert(Real_Vatrain.numpy(), bus_slack, values = 0, axis=1)
# Real_Vtrain = Real_Vmtrain.numpy()* np.exp(1j *Real_Vatrain)
# Real_Ptrain, Real_Qtrain = get_PQ(Real_Vtrain)
# Real_PQtrain = np.concatenate((Real_Ptrain, Real_Qtrain), axis = 1)
# Real_PQtrain = torch.from_numpy(Real_PQtrain)

# # train load generation for generation constraints
# Real_Pdtrain = RPd[0: Ntrain]/baseMVA
# Real_Qdtrain = RQd[0: Ntrain]/baseMVA
# Real_PQdtrain = np.concatenate((Real_Pdtrain, Real_Qdtrain), axis = 1)

# incidance matricx of branch from
finc = np.zeros((branch.shape[0], Nbus), dtype = float)
tinc = np.zeros((branch.shape[0], Nbus), dtype = float)
for i in range(branch.shape[0]):
    finc[i,  branch[i, 0]-1] = 1
    tinc[i,  branch[i, 1]-1] = 1
# real Vm Va for testing samples
yvmtests = yvmtest/scale_vm*(VmUb - VmLb) + VmLb
yvatests = yvatest/scale_va
    
# Va
Real_Va = yvatests.clone().numpy()
Real_Va = np.insert(Real_Va, bus_slack, values = 0, axis=1)

# V
Real_V = yvmtests.numpy() * np.exp(1j *Real_Va)
# Pg QG
Real_Pg, Real_Qg, Real_Pd, Real_Qd = get_genload(Real_V, Pdtest, Qdtest, bus_Pg, bus_Qg) 

# Jocobian matrix for Pg Qg violation
dPbus_dV,dQbus_dV = dPQbus_dV(his_V,bus_Pg,bus_Qg)

# Jocobian matrix for branch flow violation
bus_Va = np.delete(np.arange(Nbus),bus_slack)
dPfbus_dV, dQfbus_dV = dSlbus_dV(his_V,bus_Va)

# load trained model if it is testing
if flag_test == 1:
    # load trained model 
    print('load trained model: model_vm_load')
    model_vm = NetVm(input_channels,output_channels_vm,hidden_units,khidden_Vm)
    model_vm.load_state_dict(torch.load(PATHVm, map_location=device))
    model_vm.eval()
    model_vm.to(device)       
    # load trained model 
    print('load trained model: model_va_load')
    model_va = NetVa(input_channels,output_channels_va,hidden_units,khidden_Vm)
    model_va.load_state_dict(torch.load(PATHVa, map_location=device))
    model_va.eval() 
    model_va.to(device)   

In [ ]:
print('*'*5+'begin repeated calcualtion for time testing'+'*'*5)  
Pred_timeVm_NN_log = np.zeros(REPEAT)
Pred_timeVa_NN_log = np.zeros(REPEAT)
Pred_PQgtime_log = np.zeros((REPEAT,2))
Pred_timeVm_log = np.zeros((REPEAT,2))
Pred_timeVa_log = np.zeros((REPEAT,2))
for k in range(REPEAT):
    if (k+1)%10 == 0:
        print('REPEAT',k+1)
            
    # Predicted data
    time_PredVm_NN = 0
    yvmtest_hat = torch.zeros((Ntest, Nbus))
    for step, (test_x, test_y) in enumerate(test_loader_vm):
        test_x = test_x.to(device) 
        start_time = time.process_time()
        yvmtest_hat[step] = model_vm(test_x)
        delta_time = time.process_time() - start_time
        time_PredVm_NN = time_PredVm_NN + delta_time
    
    yvmtest_hat = yvmtest_hat.cpu()
    start_time = time.process_time()
    yvmtest_hats = yvmtest_hat.detach()/scale_vm*(VmUb - VmLb) + VmLb
    yvmtest_hat_clip = get_clamp(yvmtest_hats, hisVm_min, hisVm_max)
    time_PredVm = time.process_time() - start_time
    time_PredVm = time_PredVm + time_PredVm_NN

    time_PredVa_NN = 0
    yvatest_hat = torch.zeros((Ntest, Nbus-1))
    for step, (test_x, test_y) in enumerate(test_loader_va):
        test_x = test_x.to(device)
        start_time = time.process_time()
        yvatest_hat[step] = model_va(test_x)
        delta_time = time.process_time() - start_time
        time_PredVa_NN = time_PredVa_NN + delta_time

    yvatest_hat = yvatest_hat.cpu()
    start_time = time.process_time()
    yvatest_hats = yvatest_hat.detach()/scale_va
    time_PredVa = time.process_time() - start_time
    time_PredVa = time_PredVa + time_PredVa_NN

    # Va
    Pred_Va = yvatest_hats.clone().numpy()
    Pred_Va = np.insert(Pred_Va, bus_slack, values = 0, axis=1)
    
    ## calculate Pg QG
    start_time = time.process_time()
    Pred_V = yvmtest_hat_clip.clone().numpy()* np.exp(1j * Pred_Va)  # predicted V
    Pred_Pg, Pred_Qg, Pred_Pd, Pred_Qd = get_genload(Pred_V, Pdtest, Qdtest, bus_Pg, bus_Qg)
    Pred_PQgtime = time.process_time() - start_time    
    # Pg Qg violation
    lsPg,lsQg,lsidxPg,lsidxQg,vio_PQgmaxmin,vio_PQg,deltaPgL,deltaPgU,deltaQgL,deltaQgU = get_vioPQg(Pred_Pg, bus_Pg, MAXMIN_Pg, Pred_Qg, bus_Qg, MAXMIN_Qg)
    lsidxPQg = np.squeeze(np.array(np.where((lsidxPg + lsidxQg) > 0)))
    num_viotest = np.size(lsidxPQg) # number of violated samples

    # revise power flow (from)
    vio_branang,vio_branpf,deltapf,vio_branpfidx,lsSf,_,lsSf_sampidx,_ = get_viobran2(Pred_V,Pred_Va,branch,Yf,Yt)
    vio_branpf_num = np.size(np.where(vio_branpfidx>0))
    lsSf_sampidx = np.asarray(lsSf_sampidx)

    # post-processing of Vm Va 
    Pred_Va1 = Pred_Va.copy()
    Pred_Vm1 = yvmtest_hat_clip.clone().numpy()

    start_time = time.process_time()
    if flag_hisv:
        print('\nUse historical V to calculate dV')
        dV1 = get_hisdV(lsPg,lsQg,lsidxPg,lsidxQg,num_viotest,k_dV,bus_Pg,bus_Qg,dPbus_dV,dQbus_dV)
    else:
        print('\nUse predicted V to calculate dV')
        dV1 = get_dV(Pred_V,lsPg,lsQg,lsidxPg,lsidxQg,num_viotest,k_dV,bus_Pg,bus_Qg)

    if vio_branpf_num > 0:
        dV_branch = np.zeros((lsSf_sampidx.shape[0], Nbus*2))
        start_time = time.process_time()
        for i in range(lsSf_sampidx.shape[0]):
            # Pl/deltaSl
            mp = np.array(lsSf[i][:, 2]/lsSf[i][:, 1]).reshape(-1, 1)
            mq = np.array(lsSf[i][:, 3]/lsSf[i][:, 1]).reshape(-1, 1)
            # dPf_dVaVm
            dPdV = dPfbus_dV[np.array(lsSf[i][:, 0].astype(int)).squeeze(), :] 
            # dQf_dVaVm
            dQdV = dQfbus_dV[np.array(lsSf[i][:, 0].astype(int)).squeeze(), :]
            dmp = mp*dPdV 
            dmq = mq*dQdV
            # M: dS_dVaVm
            dmpq_inv = np.linalg.pinv(dmp+dmq)
            dV_branch[i] = np.dot(dmpq_inv, np.array(lsSf[i][:, 1])).squeeze()
        dV1 = dV1 + dV_branch

    # dV
    Pred_Va1[lsidxPQg, :] = Pred_Va[lsidxPQg, :] - dV1[:, 0:Nbus]
    Pred_Va1[:,bus_slack] = 0
    Pred_Vm1[lsidxPQg, :] = yvmtest_hat_clip.numpy()[lsidxPQg, :] - dV1[:, Nbus:2*Nbus]
    Pred_Vm1_clip = get_clamp(torch.from_numpy(Pred_Vm1), hisVm_min, hisVm_max)
    Pred_V1 = Pred_Vm1_clip.numpy() * np.exp(1j*Pred_Va1) # revised V
    Pred_Pg1, Pred_Qg1, Pred_Pd1, Pred_Qd1 = get_genload(Pred_V1, Pdtest, Qdtest, bus_Pg, bus_Qg)
    Pred_PQgtime1 = time.process_time() - start_time

    # save time
    Pred_timeVm_NN_log[k] = time_PredVm_NN/Ntest
    Pred_timeVa_NN_log[k] = time_PredVa_NN/Ntest
    Pred_PQgtime_log[k,0] = Pred_PQgtime/Ntest
    Pred_PQgtime_log[k,1] = Pred_PQgtime1/Ntest
    Pred_timeVm_log[k,0] = (Pred_PQgtime + time_PredVm)/Ntest
    Pred_timeVa_log[k,0] = (Pred_PQgtime + time_PredVa)/Ntest
    Pred_timeVm_log[k,1] = (Pred_PQgtime + Pred_PQgtime1 + time_PredVm)/Ntest
    Pred_timeVa_log[k,1] = (Pred_PQgtime + Pred_PQgtime1 + time_PredVa)/Ntest

print('*'*5+'end repeated calcualtion for time testing'+'*'*5) 

# performance evaluation
# no revison
mae_Vmtest = get_mae(yvmtests, yvmtest_hat_clip.detach())
mre_Vmtest_clip = get_rerr(yvmtests,yvmtest_hat_clip.detach())
mae_Vatest = get_mae(yvatests, yvatest_hats)
mre_Vatest = get_rerr(yvatests, yvatest_hats)
# after post-processing
mae_Vmtest1 = get_mae(yvmtests, Pred_Vm1_clip)
mae_Vatest1 = get_mae(torch.from_numpy(Real_Va).float(), torch.from_numpy(Pred_Va1).float())

# load satisfaction
mre_Pd = get_rerr(torch.from_numpy(Real_Pd.sum(axis=1)), torch.from_numpy(Pred_Pd.sum(axis=1)))
mre_Qd = get_rerr(torch.from_numpy(Real_Qd.sum(axis=1)), torch.from_numpy(Pred_Qd.sum(axis=1)))
mre_Pd1 = get_rerr(torch.from_numpy(Real_Pd.sum(axis=1)), torch.from_numpy(Pred_Pd1.sum(axis=1)))
mre_Qd1 = get_rerr(torch.from_numpy(Real_Qd.sum(axis=1)), torch.from_numpy(Pred_Qd1.sum(axis=1)))

# optimality loss
# no revison
Pred_cost = get_Pgcost(Pred_Pg,idxPg,gencost)
Real_cost = get_Pgcost(Real_Pg,idxPg,gencost)
mre_cost = get_rerr2(torch.from_numpy(Real_cost), torch.from_numpy(Pred_cost))
# after post-processing
Pred_cost1 = get_Pgcost(Pred_Pg1,idxPg,gencost)
mre_cost1 = get_rerr2(torch.from_numpy(Real_cost), torch.from_numpy(Pred_cost1))

# branch violation
vio_branang,vio_branpf,deltapf = get_viobran(Pred_V,Pred_Va,branch,Yf,Yt)
if deltapf.shape[1] > 2:
    res_branpf = np.array((np.mean(deltapf[:,1]),np.max(deltapf[:,1]),np.mean(deltapf[:,2]),np.max(deltapf[:,2])))
else:
    res_branpf = np.array((np.mean(deltapf[:,1]),np.max(deltapf[:,1])))

# Pg Qg violation degree
res_PQUL = np.zeros((2, 8))
res_PQUL[0] = np.array((np.mean(deltaPgL[:,1]),np.min(deltaPgL[:,1]),np.mean(deltaPgU[:,1]),np.max(deltaPgU[:,1]), \
                          np.mean(deltaQgL[:,1]),np.min(deltaQgL[:,1]),np.mean(deltaQgU[:,1]),np.max(deltaQgU[:,1])))

# Pg Qg violation after post-processing
_,_,lsidxPg1,lsidxQg1,vio_PQgmaxmin1,vio_PQg1,deltaPgL1,deltaPgU1,deltaQgL1,deltaQgU1 = get_vioPQg(Pred_Pg1, bus_Pg, MAXMIN_Pg, Pred_Qg1, bus_Qg, MAXMIN_Qg)
lsidxPQg1 = np.squeeze(np.array(np.where(lsidxPg1 + lsidxQg1> 0)))
num_viotest1 = np.size(lsidxPQg)
# Pg Qg violation degree after revision
res_PQUL[1] = np.array((np.mean(deltaPgL1[:,1]),np.min(deltaPgL1[:,1]),np.mean(deltaPgU1[:,1]),np.max(deltaPgU1[:,1]), \
                          np.mean(deltaQgL1[:,1]),np.min(deltaQgL1[:,1]),np.mean(deltaQgU1[:,1]),np.max(deltaQgU1[:,1])))

# branch violation after post-processing
vio_branang1,vio_branpf1,deltapf1 = get_viobran(Pred_V1,Pred_Va1,branch,Yf,Yt)
if deltapf1.shape[1] > 2:
    res_branpf1 = np.array((np.mean(deltapf1[:,1]),np.max(deltapf1[:,1]),np.mean(deltapf1[:,2]),np.max(deltapf1[:,2])))
else:
    res_branpf1 = np.array((np.mean(deltapf1[:,1]),np.max(deltapf1[:,1])))

# save results
resvio = torch.zeros(2, 7)
resvio1 = torch.zeros(2, 7)
resvio[0] = torch.tensor([torch.mean(mre_cost),torch.mean(mre_Pd),torch.mean(mre_Qd), torch.mean(vio_PQg[:, 0]), torch.mean(vio_PQg[:, 1]), \
                         torch.mean(vio_branang), torch.mean(vio_branpf)]) 
resvio[1] = torch.tensor([torch.mean(mre_cost1),torch.mean(mre_Pd1),torch.mean(mre_Qd1), torch.mean(vio_PQg1[:, 0]), torch.mean(vio_PQg1[:, 1]), \
                         torch.mean(vio_branang1), torch.mean(vio_branpf1)]) 
resvio1[0] = torch.tensor([torch.max(mre_cost),torch.max(mre_Pd),torch.max(mre_Qd), torch.min(vio_PQg[:, 0]), torch.min(vio_PQg[:, 1]), \
                         torch.min(vio_branang), torch.min(vio_branpf)])
resvio1[1] = torch.tensor([torch.max(mre_cost1),torch.max(mre_Pd1),torch.max(mre_Qd1), torch.min(vio_PQg1[:, 0]), torch.min(vio_PQg1[:, 1]), \
                         torch.min(vio_branang1), torch.min(vio_branpf1)]) 

resvio = np.around(resvio.numpy(), decimals=2)
resvio1 = np.around(resvio1.numpy(), decimals=2)
maeV = np.double(torch.tensor([[mae_Vmtest, mae_Vatest],[mae_Vmtest1, mae_Vatest1]]).numpy())
lossvm = np.array(lossvm)
lossva = np.array(lossva)
PredtimeVm = np.mean(Pred_timeVm_log, axis=0)
PredtimeVa = np.mean(Pred_timeVa_log, axis=0)
mre_cost = np.array(mre_cost)
mre_cost1 = np.array(mre_cost1)
scipy.io.savemat(resultnm, \
mdict={'resvio': resvio,'resvio1': resvio1,'maeV': maeV,'res_PQUL': res_PQUL,'deltaPgL': deltaPgL,'deltaPgU': deltaPgU,'deltaQgL': deltaQgL,'deltaQgU': deltaQgU, \
    'PredtimeVm': PredtimeVm,'PredtimeVa': PredtimeVa,'Pred_timeVm_log': Pred_timeVm_log,'Pred_timeVa_log': Pred_timeVa_log, \
    'deltaPgL1': deltaPgL1,'deltaPgU1': deltaPgU1,'deltaQgL1': deltaQgL1,'deltaQgU1': deltaQgU1,'deltapf':deltapf,'deltapf1':deltapf1, \
    'res_branpf':res_branpf,'res_branpf1':res_branpf1,'mre_cost':mre_cost,'mre_cost1':mre_cost1,'Pred_timeVm_NN_log':Pred_timeVm_NN_log, \
    'Pred_timeVa_NN_log':Pred_timeVa_NN_log,'Pred_PQgtime_log':Pred_PQgtime_log
})

In [ ]:
print('***************Summary result case',Nbus, 'sys_R', sys_R,'***************')
print('training setting: Ntrain',Ntrain,' Ntest',Ntest, 'batch_size',batch_size_training,\
      'Lm', Lm,'La', La,' Lrm', Lrm,' Lra', Lra,' EpochVm',EpochVm,'scale_va',scale_va,'  scale_vm',scale_vm)
print('NN layer: hidden_units',hidden_units,' khidden_Vm',khidden_Vm*hidden_units ,'khidden_Va',khidden_Va*hidden_units )
print(model_vm)
print(model_va)
print('\n','*'*20,'before modification: ','*'*20)
print('mae_Vmtest', mae_Vmtest,'mae_Vatest', mae_Vatest)
print('num_viotest', num_viotest)
print('vio_Pgmax: mean', torch.mean(vio_PQgmaxmin[:,0]).numpy(), '%  max', torch.max(vio_PQgmaxmin[:,0]).numpy(), \
      'vio_Pgmin: mean',torch.mean(vio_PQgmaxmin[:,1]).numpy(), '%  max', torch.max(vio_PQgmaxmin[:,1]).numpy())
print('vio_Qgmax: mean', torch.mean(vio_PQgmaxmin[:,2]).numpy(), '%  max', torch.max(vio_PQgmaxmin[:,2]), \
      'vio_Qgmin: mean',torch.mean(vio_PQgmaxmin[:,3]).numpy(), '%  max', torch.max(vio_PQgmaxmin[:,3]).numpy())
print('vio_Pgmaxmin: mean', torch.mean(vio_PQg[:, 0]).numpy(), '%  max', torch.max(vio_PQg[:, 0]).numpy(), \
      'vio_Qgmaxmin: mean',torch.mean(vio_PQg[:, 1]).numpy(), '%  max', torch.max(vio_PQg[:, 1]).numpy())
print('mean', torch.mean(vio_branang).numpy(), '%  max', torch.max(vio_branang).numpy(),'% ', \
      'vio_branpf: mean',torch.mean(vio_branpf).numpy(), '%  max', torch.max(vio_branpf).numpy(),'% ')
print('mean mre_cost', resvio[0,0],'%  ','mre_Pd',resvio[0,1],'%  ','mre_Qd',resvio[0,2],'vio_PQg',resvio[0,3],'%  ', \
      'vio_PQg',resvio[0,4],'%  ','vio_branang',resvio[0,5],'%  ', 'vio_branpf',resvio[0,6],'%  ')

print('\n','*'*20,'after post-processing: ','*'*20)
print('mae_Vmtest1', mae_Vmtest1,'mae_Vatest1', mae_Vatest1)
print('num_viotest1', num_viotest1) 
print('vio_Pgmax: mean', torch.mean(vio_PQgmaxmin1[:,0]).numpy(), '%  max', torch.max(vio_PQgmaxmin1[:,0]).numpy(), \
      'vio_Pgmin: mean',torch.mean(vio_PQgmaxmin1[:,1]).numpy(), '%  max', torch.max(vio_PQgmaxmin1[:,1]).numpy())
print('vio_Qgmax: mean', torch.mean(vio_PQgmaxmin1[:,2]).numpy(), '%  max', torch.max(vio_PQgmaxmin1[:,2]).numpy(), \
      'vio_Qgmin: mean',torch.mean(vio_PQgmaxmin1[:,3]).numpy(), '%  max', torch.max(vio_PQgmaxmin1[:,3]).numpy())
print('vio_Pgmaxmin: mean', torch.mean(vio_PQg1[:, 0]).numpy(), '%  max', torch.max(vio_PQg1[:, 0]).numpy(), \
      'vio_Qgmaxmin: mean',torch.mean(vio_PQg1[:, 1]).numpy(), '%  max', torch.max(vio_PQg1[:, 1]).numpy())
print('vio_branang: mean', torch.mean(vio_branang1).numpy(), '%  max', torch.max(vio_branang1).numpy(),'% ', \
      'vio_branpf: mean',torch.mean(vio_branpf1).numpy(), '%  max', torch.max(vio_branpf1).numpy(),'% ')
print('mean mre_cost', resvio[1,0],'%  ','mre_Pd',resvio[1,1],'%  ','mre_Qd',resvio[1,2],'%  ', \
      'vio_PQg',resvio[1,3],'%  ','vio_PQg',resvio[1,4],'%  ','vio_branang',resvio1[1,5],'%  ', 'vio_branpf',resvio[1,6],'%  ')

if flag_test == 0:
    print('\n', '*'*10,'trian vs test','*'*10)
    print('voltage')
    print('mae_Vmtrain', mae_Vmtrain)
    print('mre_Vmtrain_clip', torch.mean(mre_Vmtrain_clip),'%  ',  torch.max(mre_Vmtrain_clip),'%')
    print('mae_Vmtest', mae_Vmtest)
    print('mre_Vmtest_clip', torch.mean(mre_Vmtest_clip),'%  ',  torch.max(mre_Vmtest_clip),'%')
    print('angle')
    print('mae_Vatrain', mae_Vatrain)
    print('mre_Vatrain', torch.mean(mre_Vatrain),'%  ',  torch.max(mre_Vatrain),'%')
    print('mae_Vatest', mae_Vatest)
    print('mre_Vatest', torch.mean(mre_Vatest),'%  ',  torch.max(mre_Vatest),'%')
    print('time_trianVm',round(time_trianVm,6),'sec  time_trianVa',round(time_trianVa,6), \
          'sec  \ntime_trianVm',round(time_trianVm/60,6),'min  time_trianVa',round(time_trianVa/60,6),'min  \n')
    
print('\n','*'*20,'computation time','*'*20)
print('Pred_time for each sample:\nPred_timeVm',np.round(PredtimeVm[0],7),'sec\nPred_timeVa',np.round(PredtimeVa[0],7),'sec')
print('PredtimeVa',PredtimeVa,'PredtimeVm',PredtimeVm)